# Machine Learning in Business - Course Project
We work for the OilyGiant mining company. Our task is to find the best place for a new well.
Steps to choose the location:
- Collecting the oil well parameters in the selected region: oil quality and volume of reserves;
- Building a model for predicting the volume of reserves in the new wells;
- Picking the oil wells with the highest estimated values;
- Picking the region with the highest total profit for the selected oil wells.

We have data on oil samples from three regions. Parameters of each oil well in the region are already known. We will build a model that will help to pick the region with the highest profit margin. Then, we will analyze potential profit and risks using the Bootstrapping technique.

## Model

Import necessary modules.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import plotly.express as px

Load data.

In [2]:
try:
    df0 = pd.read_csv('geo_data_0.csv')
    df1 = pd.read_csv('geo_data_1.csv')
    df2 = pd.read_csv('geo_data_2.csv')
except:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
    df2 = pd.read_csv('/datasets/geo_data_2.csv')

Split features and target, then training and validation sets.

In [3]:
x0 = df0[['f0', 'f1', 'f2']]
x1 = df1[['f0', 'f1', 'f2']]
x2 = df2[['f0', 'f1', 'f2']]

y0 = df0['product']
y1 = df1['product']
y2 = df2['product']

x0_train, x0_valid, y0_train, y0_valid = train_test_split(x0, y0, test_size=0.25, random_state=42069)
x1_train, x1_valid, y1_train, y1_valid = train_test_split(x1, y1, test_size=0.25, random_state=42069)
x2_train, x2_valid, y2_train, y2_valid = train_test_split(x2, y2, test_size=0.25, random_state=42069)

Create and fit a scaler for the features.

In [4]:
scaler = StandardScaler()
scaler.fit(pd.concat([x0, x1, x2]))

StandardScaler()

Train, predict and evaluate a model for each dataset.

In [5]:
model0 = LinearRegression()
model0.fit(scaler.transform(x0_train), y0_train)
y0_predicted = model0.predict(scaler.transform(x0_valid))
print('Prediction average:', y0_predicted.mean())
print('RMSE:', mean_squared_error(y0_predicted, y0_valid, squared=False))
print('Sanity check RMSE:', mean_squared_error(pd.Series(y0_train.mean(), index=y0_valid.index), y0_valid, squared=False))

Prediction average: 92.47745883135654
RMSE: 37.50962222969333
Sanity check: 44.19041951753856


In [6]:
model1 = LinearRegression()
model1.fit(scaler.transform(x1_train), y1_train)
y1_predicted = model1.predict(scaler.transform(x1_valid))
print('Prediction average:', y1_predicted.mean())
print('RMSE:', mean_squared_error(y1_predicted, y1_valid, squared=False))
print('Sanity check RMSE:', mean_squared_error(pd.Series(y1_train.mean(), index=y1_valid.index), y1_valid, squared=False))

Prediction average: 68.75615816904572
RMSE: 0.8907321858616108
Sanity check: 45.7906709311826


In [7]:
model2 = LinearRegression()
model2.fit(scaler.transform(x2_train), y2_train)
y2_predicted = model2.predict(scaler.transform(x2_valid))
print('Prediction average:', y2_predicted.mean())
print('RMSE:', mean_squared_error(y2_predicted, y2_valid, squared=False))
print('Sanity check RMSE:', mean_squared_error(pd.Series(y2_train.mean(), index=y2_valid.index), y2_valid, squared=False))

Prediction average: 94.73914527032773
RMSE: 40.00730730575776
Sanity check: 44.76476047372208


Regions 0 and 2 have similar average production and RMSE score, that is only a few units lower than the RMSE score of the sanity check model. Of the two, region 2 has the higher production average and higher (worse) RMSE score. On the other hand, the average production in region 1 is much lower, but the prediction error is less than one unit, making it a lot easier to pick the best sites for development.

## Profit Calculation

Find the minimum average production per well required to make profit.

In [8]:
investment = 10**8
n_well = 200
unit_price = 4500

investment/n_well/unit_price

111.11111111111111

This is higher than any predicted average. In reality, only 200 wells with the best prospects out 500 will be developed. We'll use the bootstraping technique to evaluate profit with a 95% confidence interval.

In [16]:
def revenue(actual, predicted, n):
    pred_sorted = predicted.sort_values(ascending=False)
    selected = actual[pred_sorted.index][:n]
    return selected.sum() * unit_price - investment

state = np.random.RandomState(12345)
    
def bootstrap(target, probabilities, n_sample, n_select, reps=1000):
    values = []
    for i in range(reps):
        target_subsample = target.sample(n_sample, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]

        values.append(revenue(target_subsample, probs_subsample, n_select))

    return pd.Series(values)


In [17]:
bs_result0 = bootstrap(y0_valid.reset_index(drop=True), pd.Series(y0_predicted), 500, n_well)

In [18]:
bs_result1 = bootstrap(y1_valid.reset_index(drop=True), pd.Series(y1_predicted), 500, n_well)

In [19]:
bs_result2 = bootstrap(y2_valid.reset_index(drop=True), pd.Series(y2_predicted), 500, n_well)

In [20]:
results = pd.DataFrame(dict(
    region = ['0', '1', '2'],
    mean = [result.mean() for result in [bs_result0, bs_result1, bs_result2]],
    interval_high = [result.quantile(0.975) for result in [bs_result0, bs_result1, bs_result2]],
    interval_low = [result.quantile(0.025) for result in [bs_result0, bs_result1, bs_result2]],
    loss_risk_percent = [(result < 0).sum() / len(result) * 100 for result in [bs_result0, bs_result1, bs_result2]]
))
results.astype({'mean': int, 'interval_high': int, 'interval_low': int})

,region,mean,interval_high,interval_low,loss_risk_percent
0,0,4797717,10020736,-520723,3.8
1,1,4704893,9129342,460629,0.9
2,2,3816509,9072061,-1893091,8.7


## Conclusion

In [21]:
results['err_plus'] = results['interval_high'] - results['mean']
results['err_minus'] = results['mean'] - results['interval_low']

In [ ]:
fig = px.scatter(
    results, 
    x="region", y="mean", color="region",
    error_y="err_plus", error_y_minus="err_minus",
    title='Projected Profit by Region<br><sup>Confidence Interval: 95%</sup>',
    labels={'region': 'Region', 'mean': 'Profit (USD)', 'interval_high': '97.5th percentile', 'interval_low': '2.5th percentile'},
    hover_data=['interval_high', 'interval_low'],
    width=420,
)
fig.add_hline(y=0, line_width=3, line_dash="dash", line_color="black")
fig.update_traces(marker=dict(size=18, symbol='diamond'), error_y=dict(width=10, thickness=3), showlegend=False)
fig.show()

We are required to consider only the regions where the risk of losses is lower than 2.5%. From the ones that fit the criteria, the region with the highest average profit should be selected. Only region 1 has a risk of loss low enough (0.9%) so it is the region we recommand for development. Despite having the lowest average production by a large margin, well production prediction is much more accurate in this region, as seen in the low RMSE value. This makes it the safest option.